In [214]:
# import libraries

import os
import numpy as np
import pandas as pd
import urllib.parse
from urllib.parse import unquote


### Creating the Dataframe for the LMQL-Pipeline

In [261]:
data_dir = os.path.join(os.getcwd(), 'wikispeedia_data', 'wikispeedia_paths-and-graph')

In [262]:
# decode url text inside page names
def decode_list(url_encoded_list):
    """Decodes url_encoded path.
    
    Args:
        url_encoded_list (list): list of url_encoded page names
    
    Returns:
        list: list of decoded page names
    """
    return [urllib.parse.unquote(element).replace(',', '') for element in url_encoded_list]

# pull in finished games and prepare for append
paths_finished = pd.read_csv(os.path.join(data_dir, 'paths_finished.tsv'), sep='\t', skiprows = 16, header = None, names = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'])
paths_finished['path_decoded'] = paths_finished['path'].str.split(';').apply(decode_list)
paths_finished['origin'] = paths_finished['path_decoded'].str[0]
paths_finished['goal'] = paths_finished['path_decoded'].str[-1]
paths_finished['finished'] = 1

paths_finished



,hashedIpAddress,timestamp,durationInSec,path,rating,path_decoded,origin,goal,finished
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,1
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,1
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,1
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN,"[14th_century, Renaissance, Ancient_Greece, Gr...",14th_century,Greece,1
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,1
...,...,...,...,...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NaN,"[Yagan, Ancient_Egypt, Civilization]",Yagan,Civilization,1
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3.0,"[Yagan, Folklore, Brothers_Grimm, <, 19th_cent...",Yagan,Fiction,1
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NaN,"[Yagan, Australia, England, France, United_Sta...",Yagan,U.S._Open_(tennis),1
51316,19f8284371753362,1298792567,56,Yarralumla%2C_Australian_Capital_Territory;Aus...,1.0,"[Yarralumla_Australian_Capital_Territory, Aust...",Yarralumla_Australian_Capital_Territory,Abraham_Lincoln,1


In [263]:
# for testing, let's only take the top 50 
all_paths = paths_finished
all_paths['game_pair'] = all_paths['origin'] + "->" + all_paths['goal']
game_pair_counts = all_paths['game_pair'].value_counts().rename_axis('game_pair').reset_index(name='count')
game_pair_counts = game_pair_counts.rename(columns={'count': 'human_attempts'})

game_pair_cands = game_pair_counts[:50]

In [264]:
def all_to_candidates(
    all_paths: pd.DataFrame,
    candidates: pd.DataFrame,
    new_var_pre_agg: str,
    new_var_post_agg: str,
    finished_only: bool
) -> pd.DataFrame:    
    """For mapping all_path data to candidate set
    
    Args:
        all_paths (pd.DataFrame): all paths data
        candidates (pd.DataFrame): candidate set
        new_var_pre_agg (str): variable to aggregate
        new_var_post_agg (str): name of new variable
        finished_only (bool): whether to aggregate only finished games
    Returns:
        pd.DataFrame: candidates with new variable
    """
    if finished_only == True:
        all_pairs_avg = all_paths[all_paths['finished'] == 1].groupby('game_pair')[new_var_pre_agg].agg('mean').reset_index().copy()
    else:
        all_pairs_avg = all_paths.groupby('game_pair')[new_var_pre_agg].agg('mean').reset_index().copy()
    all_pairs_avg = all_pairs_avg.rename(columns={new_var_pre_agg:new_var_post_agg})
    return candidates.merge(right=all_pairs_avg, on='game_pair').copy()

In [266]:
## Average rounds-to-completion
all_paths['game_length'] = all_paths['path_decoded'].apply(len)
all_paths.loc[all_paths['finished'] == 0, 'game_length'] = np.nan
game_pair_cands_oth = all_to_candidates(all_paths, game_pair_cands, 'game_length', 'avg_game_length', True)

game_pair_cands_oth[['origin', 'target']] = game_pair_cands_oth['game_pair'].str.split('->', expand=True)


In [220]:
game_pair_cands_oth.to_csv('pipeline_dataset.csv', columns=['game_pair', 'origin', 'target'], index=False)

## RESULTS OF THE PIPELINE

In [267]:
import pickle

# Load 1
with open('Mistral_Games.pickle', 'rb') as file:
    Mistral_Games = pickle.load(file)


In [268]:
Mistral_Games

{'Asteroid->Viking': {'rep0': ['Comet_Shoemaker-Levy_9',
   'Magnesium',
   'Scotland',
   'Norway',
   'Viking'],
  'rep1': ['Venus', 'Nitrogen', 'Oxygen', 'Sweden', 'Viking'],
  'rep2': ['Comet_Shoemaker-Levy_9',
   'Moon',
   'Mars',
   'European_Space_Agency',
   'Norway',
   'Viking'],
  'rep3': ['Venus', 'European_Space_Agency', 'Norway', 'Viking'],
  'rep4': ['Venus',
   'Nitrogen',
   'Natural_gas',
   'September_11,_2001_attacks',
   'Steel',
   'Vanadium',
   'Nuclear_fission',
   'Denmark',
   'Viking'],
  'rep5': ['Venus',
   'Nitrogen',
   'Natural_gas',
   'Ocean',
   'Norse_mythology',
   'Denmark',
   'Viking'],
  'rep6': ['Venus',
   'Nitrogen',
   'Nitrogen_fixation',
   "Earth's_atmosphere",
   'Planet',
   'Nitrogen',
   "Earth's_atmosphere",
   'Planet',
   'Scattered_disc',
   'Solar_System',
   'Scattered_disc',
   'Solar_System',
   'Nickel',
   'European_Union',
   'Norway',
   'Viking'],
  'rep7': ['Carl_Friedrich_Gauss', 'Denmark', 'Viking'],
  'rep8': ['1_Ce

**NOTE THAT**: the results are contained in a big nested dictionary of the following structure:

{   'game_pair0': {
        rep0 = [steps], rep1 = [steps], ..., rep9 = [steps]
    },

    game_pair1': {
        rep0 = [steps], rep1 = [steps], ..., rep9 = [steps]
    },

    ...

    game_pair49': {
        rep0 = [steps], rep1 = [steps], ..., rep9 = [steps]
    },
    
}


Below is an example of a opossible laoding as pandas dataframe, which howver results in a sparse df.

In [272]:
data = []

for game_pair, reps in Mistral_Games.items():
    for rep, values in reps.items():
        data.append([game_pair, rep] + values)

df_results = pd.DataFrame(data)
df_results.columns = ['Game Pair', 'Repetition'] + [f'Value_{i}' for i in range(df_results.shape[1] - 2)]

df_results.head()

,Game Pair,Repetition,Value_0,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,...,Value_10,Value_11,Value_12,Value_13,Value_14,Value_15,Value_16,Value_17,Value_18,Value_19
0,Asteroid->Viking,rep0,Comet_Shoemaker-Levy_9,Magnesium,Scotland,Norway,Viking,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Asteroid->Viking,rep1,Venus,Nitrogen,Oxygen,Sweden,Viking,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Asteroid->Viking,rep2,Comet_Shoemaker-Levy_9,Moon,Mars,European_Space_Agency,Norway,Viking,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Asteroid->Viking,rep3,Venus,European_Space_Agency,Norway,Viking,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Asteroid->Viking,rep4,Venus,Nitrogen,Natural_gas,"September_11,_2001_attacks",Steel,Vanadium,Nuclear_fission,Denmark,...,None,None,None,None,None,None,None,None,None,None
